# Clustering Crypto

In [35]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [36]:
# Load the crypto_data.csv dataset.
file_path="Resources/crypto_data.csv"
crypto_df=pd.read_csv(file_path,index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [37]:
crypto_df.columns

Index(['CoinName', 'Algorithm', 'IsTrading', 'ProofType', 'TotalCoinsMined',
       'TotalCoinSupply'],
      dtype='object')

In [38]:
# Keep all the cryptocurrencies that are being traded.
crypto_df.loc[(crypto_df["IsTrading"]=="True")]
crypto_df.head(5)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [39]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df["Algorithm"]=crypto_df["Algorithm"].dropna()
crypto_df.head(5)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [40]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns="IsTrading",inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [41]:
# Remove rows that have at least 1 null value.
crypto_df.dropna()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,PoS,1.283270e+05,1000000


In [42]:
# Keep the rows where coins are mined.
crypto_df=crypto_df.loc[(crypto_df["TotalCoinsMined"])>0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [43]:
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 577 entries, 42 to DVTC
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         577 non-null    object 
 1   Algorithm        577 non-null    object 
 2   ProofType        577 non-null    object 
 3   TotalCoinsMined  577 non-null    float64
 4   TotalCoinSupply  577 non-null    object 
dtypes: float64(1), object(4)
memory usage: 27.0+ KB


In [44]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_name_df=crypto_df[["CoinName"]]
crypto_name_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [45]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns="CoinName",inplace=True)
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [46]:
crypto_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [47]:
# Use get_dummies() to create variables for text features.
X=pd.get_dummies(crypto_df,columns=["Algorithm","ProofType"])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.14159E+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
# Standardize the data with StandardScaler().
data_scaler = StandardScaler()
crypto_scaled = data_scaler.fit_transform(X)
crypto_scaled[:5]

array([[-0.11450077, -0.15072488, -0.04166667, -0.04166667, -0.04166667,
        -0.05897678, -0.0934947 , -0.04166667, -0.05897678, -0.05897678,
        -0.04166667, -0.04166667, -0.18949048, -0.05897678, -0.0934947 ,
        -0.04166667, -0.11081833, -0.0722944 , -0.04166667, -0.04166667,
        -0.1518211 , -0.04166667, -0.13280318, -0.04166667, -0.04166667,
        -0.0835512 , -0.05897678, -0.04166667, -0.04166667, -0.04166667,
        -0.05897678, -0.04166667, -0.0835512 , -0.0934947 , -0.10250796,
        -0.04166667, -0.1258772 , -0.13280318, -0.1518211 , -0.04166667,
        -0.0835512 , -0.04166667, -0.04166667, -0.0722944 , -0.17423301,
        -0.04166667, -0.04166667, -0.04166667, -0.0722944 , -0.16888013,
        -0.30802055, -0.04166667, -0.0934947 , -0.0934947 , -0.05897678,
         1.39963365, -0.04166667, -0.04166667, -0.04166667, -0.0835512 ,
        -0.04166667, -0.04166667, -0.04166667, -0.04166667, -0.04166667,
        -0.05897678, -0.04166667, -0.04166667, -0.3



### Deliverable 2: Reducing Data Dimensions Using PCA

In [49]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)

In [50]:
# Create a DataFrame with the three principal components.
pca_df=pd.DataFrame(
    data=crypto_pca,columns=["PC 1","PC 2","PC 3"] , index=crypto_df.index
)
pca_df.head(10)

,PC 1,PC 2,PC 3
42,-0.322600,1.114980,-0.555878
404,-0.305257,1.115128,-0.556299
1337,2.428280,1.731118,-0.615068
BTC,-0.141981,-1.329668,0.166634
ETH,-0.137664,-2.091259,0.331064
LTC,-0.159125,-1.079930,-0.044818
DASH,-0.392906,1.244749,-0.456250
XMR,-0.146599,-2.196369,0.343199
ETC,-0.136046,-2.091361,0.331042
ZEC,-0.134792,-2.093286,0.305528


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [51]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)


C:\Users\sandi\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1039: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



In [52]:
#create elbo curve using hv plot
elbow_data = {"k": k, "inertia" : inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k" , xticks=k ,title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [53]:
# Initialize the K-Means model.
# Initialize the K-means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)





In [54]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.

#clustered_df=pd.merge(crypto_df, pca_df, how="left", on=[crypto_df.index , pca_df.index])
clustered_df=pd.concat([crypto_df, pca_df], axis=1)
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"]=crypto_name_df["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# Add the predicted class columns
clustered_df["class"] = model.labels_


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(577, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.322600,1.114980,-0.555878,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.305257,1.115128,-0.556299,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11,2.428280,1.731118,-0.615068,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.141981,-1.329668,0.166634,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.137664,-2.091259,0.331064,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.159125,-1.079930,-0.044818,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.392906,1.244749,-0.456250,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.146599,-2.196369,0.343199,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.136046,-2.091361,0.331042,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.134792,-2.093286,0.305528,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [55]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="ProofType",
    y="TotalCoinsMined",
    z="TotalCoinSupply",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    color="class",
    symbol="class",
    width=800,

    
    
    
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [56]:
clustered_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
PC 1               float64
PC 2               float64
PC 3               float64
CoinName            object
class                int32
dtype: object

In [57]:
# Create a table with tradable cryptocurrencies.
import hvplot.pandas
clustered_df.hvplot.table(columns=['CoinName','Algorithm','ProofType','TotalCoinSupply','TotalCoinsMined','class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [58]:
# Print the total number of tradable cryptocurrencies.
clustered_df["TotalCoinsMined"].sum()

2894947428359.2773

In [59]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X_cluster=clustered_df[['TotalCoinSupply','TotalCoinsMined']].copy()
data_scaler = MinMaxScaler()
X_cluster_scaled = data_scaler.fit_transform(X_cluster)
X_cluster_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585514e-03],
       [3.14159000e-01, 2.95755050e-02],
       ...,
       [2.10000000e-05, 7.37027937e-06],
       [1.00000000e-06, 1.29582244e-07],
       [1.00000000e-04, 2.17084952e-05]])

In [60]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df=pd.DataFrame(
    data=X_cluster_scaled,columns=['TotalCoinSupply','TotalCoinsMined'],index=clustered_df.index
)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE
plot_df['CoinName']=clustered_df['CoinName']
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["class"] = model.labels_


plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141590e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [61]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["class"],
    by="class",
)


:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,class)